In [ ]:
#import rioxarray for accessing data and geopandas to clip the necessary area. unlike xarray, rioxarray or rasterxarray support raster data handling.
import pandas as pd
import rioxarray as rxr
import geopandas as gpd

In [ ]:
#use rxr.open_rasterio() to open the nc NetCDF file.  
raster_ds = rxr.open_rasterio("directory\file.nc")

In [ ]:
#open the shapefile and extract the region in the shapefile you want to use.
#view the shapefile to get the exact variable names
india = gpd.read_file("directory\shapefile.shp")
kerala = india[india["name"]=="Kerala"]
kerala_geom = kerala.geometry 
kerala_geom

In [ ]:
#code to clip the shapefile into the NetCDF data
#use the first line only if coordinate reference system is not specified in the data. 
raster_ds.rio.write_crs("EPSG:4326", inplace=True)
kerala_clipped = raster_ds.rio.clip(kerala_geom)
klds = kerala_clipped.to_dataset()
klds

In [ ]:
#the data for some unclipped region will be stilled present with variables carrying values like -999. use this code to convert them into nan.
bad_data_condition = klds.data_vars["RAINFALL"] < 0
klds = klds.where(~bad_data_condition, np.nan)
klds

In [ ]:
#to convert it into a pandas dataframe and drop the nan values
kldf = pd.DataFrame(klds.to_dataframe()).dropna()
kldf